In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
import pickle
import torch.optim as optim
import copy
import random
import os

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)  


## 0. Dataset and DataLoader

In [ ]:
class WindPowerDataset(Dataset):
    def __init__(self, csv_file, wind_power_scaler=None, weather_scaler=None, save_scalers=False):
        self.data = pd.read_csv(csv_file)

        self.data = self.data.iloc[::5, :].reset_index(drop=True)

        self.original_wind_power_std = self.data.iloc[:, 2].std()
        self.original_weather_std = self.data.iloc[:, 4:12].std()

        if wind_power_scaler is None or weather_scaler is None:
            self.wind_power_scaler = MinMaxScaler()
            self.weather_scaler = MinMaxScaler()

            self.data.iloc[:, 2] = self.wind_power_scaler.fit_transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()

            self.data.iloc[:, 4:12] = self.weather_scaler.fit_transform(self.data.iloc[:, 4:12])
            
            if save_scalers:
                with open('wind_power_scaler.pkl', 'wb') as f:
                    pickle.dump(self.wind_power_scaler, f)
                with open('weather_scaler.pkl', 'wb') as f:
                    pickle.dump(self.weather_scaler, f)
        else:
            self.wind_power_scaler = wind_power_scaler
            self.weather_scaler = weather_scaler
            self.data.iloc[:, 2] = self.wind_power_scaler.transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            self.data.iloc[:, 4:12] = self.weather_scaler.transform(self.data.iloc[:, 4:12])
    
    def __len__(self):
        return len(self.data) - 312  # 288 (1440/5) + 24 (120/5)
    
    def __getitem__(self, idx):
        history_weather = self.data.iloc[idx:idx + 288, 4:12].values.astype(float)  # [288, 8]
        wind_power_history = self.data.iloc[idx:idx + 288, 2].values.astype(float)   # [288]
        future_weather = self.data.iloc[idx + 288:idx + 312, 4:12].values.astype(float)  # [24, 8]
        future_wind_power = self.data.iloc[idx + 312, 2]  # float

        return (
            #torch.tensor(history_weather, dtype=torch.float32),
            torch.tensor(wind_power_history, dtype=torch.float32),
            torch.tensor(future_weather, dtype=torch.float32),
            torch.tensor(future_wind_power, dtype=torch.float32)
        )
    
    def get_original_stds(self):
        return {
            'original_wind_power_std': self.original_wind_power_std,
            'original_weather_std': self.original_weather_std.to_dict()
        }
    
name='CAISO_zone_1_.csv'
with open('weather_scaler.pkl', 'rb') as f:
    weather_scaler = pickle.load(f)
dataset = WindPowerDataset(name, save_scalers=True)
wind_power_scaler=dataset.wind_power_scaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=32
test_split=0.2
test_size = int(len(dataset) * test_split)
train_size = len(dataset) - test_size
train_dataset = Subset(dataset, list(range(train_size)))
test_dataset = Subset(dataset, list(range(train_size, len(dataset))))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 1. preparation

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(256, 1)
        self.relu=nn.ReLU()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.relu(output)
        return output
    
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tar_model_path1 = 'wind_caiso_lstm_sigmoid_version2.pth'
tar_model1 = load_model(tar_model_path1, device)

class WindPowerPredictor(nn.Module):  
    def __init__(self):  
        super(WindPowerPredictor, self).__init__()  
        self.gru_wind = nn.GRU(input_size=1, hidden_size=128, num_layers=1, batch_first=True)  
        self.gru_weather = nn.GRU(input_size=8, hidden_size=128, num_layers=1, batch_first=True)  
        self.fc = nn.Linear(256, 1)  
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, wind_history, weather_future):  
        wind_history = wind_history.unsqueeze(-1)  
        _, hn_wind = self.gru_wind(wind_history)  
        _, hn_weather = self.gru_weather(weather_future)  
        hn_wind = hn_wind[-1, :, :]  
        hn_weather = hn_weather[-1, :, :]  
        combined = torch.cat((hn_wind, hn_weather), dim=1)  
        output = self.fc(combined)  
        output = self.sigmoid(output)
        return output  

tar_model_path2 = 'wind_gru_caiso_sigmoid_version2.pth'
tar_model2 = load_model(tar_model_path2, device)

class WindPowerPredictorV2(nn.Module):
    def __init__(self, d_model=64, nhead=4, num_layers=3, dim_feedforward=128, dropout=0.1, max_seq_len=300):
        super(WindPowerPredictorV2, self).__init__()
        self.d_model = d_model

        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)

        self.pos_encoder_wind = nn.Parameter(torch.zeros(max_seq_len, 1, d_model))
        self.pos_encoder_weather = nn.Parameter(torch.zeros(max_seq_len, 1, d_model))


        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.fc_combine = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        # wind_history: (batch_size, T1)        e.g., (32, 288)
        # weather_future: (batch_size, T2, 8)    e.g., (32, 24, 8)

        batch_size, seq_len_wind = wind_history.shape
        _, seq_len_weather, _ = weather_future.shape

        if seq_len_wind > self.pos_encoder_wind.size(0):
            raise ValueError(f"Wind sequence length {seq_len_wind} exceeds max supported length {self.pos_encoder_wind.size(0)}")
        if seq_len_weather > self.pos_encoder_weather.size(0):
            raise ValueError(f"Weather sequence length {seq_len_weather} exceeds max supported length {self.pos_encoder_weather.size(0)}")

        wind_emb = self.embedding_wind(wind_history.unsqueeze(-1))           # (B, T1, D)
        weather_emb = self.embedding_weather(weather_future)                 # (B, T2, D)

        wind_emb = wind_emb.permute(1, 0, 2)  # (T1, B, D)
        weather_emb = weather_emb.permute(1, 0, 2)  # (T2, B, D)

        wind_emb = wind_emb + self.pos_encoder_wind[:seq_len_wind]      # (T1, B, D)
        weather_emb = weather_emb + self.pos_encoder_weather[:seq_len_weather]  # (T2, B, D)

        wind_features = self.transformer_encoder(wind_emb)              # (T1, B, D)
        weather_features = self.transformer_encoder(weather_emb)        # (T2, B, D)

        wind_last = wind_features[-1, :, :]        
        weather_last = weather_features[-1, :, :]  

        combined = torch.cat((wind_last, weather_last), dim=1)  # (B, 2D)

        output = self.fc_combine(combined)  # (B, 1)
        output = self.sigmoid(output)
        return output
    
tar_model_path3 = 'wind_caiso_transformer_sigmoid_version2.pth'
tar_model3 = WindPowerPredictorV2().to(device)
state_dict = torch.load(tar_model_path3, map_location=device)
tar_model3.load_state_dict(state_dict)
tar_model3.to(device)

class TemporalConvNet_V2(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2):
        super(TemporalConvNet_V2, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]

            padding = (kernel_size - 1) * dilation_size

            conv = nn.Conv1d(
                in_channels, out_channels, kernel_size,
                stride=1, padding=padding, dilation=dilation_size  
            )
            relu = nn.ReLU()
            drop = nn.Dropout(dropout)

            if in_channels != out_channels:
                res_conv = nn.Conv1d(in_channels, out_channels, 1)
            else:
                res_conv = None

            layers.append(nn.ModuleDict({
                'conv': conv,
                'relu': relu,
                'dropout': drop,
                'res_conv': res_conv
            }))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        # x: (batch_size, in_channels, seq_len)
        for layer in self.layers:
            residual = x  

            x = layer['conv'](x)  
            x = layer['relu'](x)
            x = layer['dropout'](x)
            x = x[:, :, :residual.size(2)]  

            # 残差连接
            if layer['res_conv'] is not None:
                residual = layer['res_conv'](residual)
            x = x + residual
            x = layer['relu'](x)  
        return x


class WindPowerPredictorTCN_V2(nn.Module):
    def __init__(self, d_model=64, dropout=0.2, fusion_mode='concat'):
        super(WindPowerPredictorTCN_V2, self).__init__()
        self.fusion_mode = fusion_mode  # 'concat' or 'add'
        self.tcn_wind = TemporalConvNet_V2(
            num_inputs=1,
            num_channels=[d_model, d_model*2, d_model, d_model],  
            kernel_size=3,
            dropout=dropout
        )

        self.tcn_weather = TemporalConvNet_V2(
            num_inputs=8,
            num_channels=[d_model, d_model*2, d_model, d_model],
            kernel_size=3,
            dropout=dropout
        )

        if fusion_mode == 'concat':
            fc_input_dim = d_model * 2
        elif fusion_mode == 'add':
            fc_input_dim = d_model
        else:
            raise ValueError("fusion_mode must be 'concat' or 'add'")

        self.fc = nn.Sequential(
            nn.Linear(fc_input_dim, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        # wind_history: (batch_size, 288)
        # weather_future: (batch_size, 24, 8)

        x_wind = wind_history.unsqueeze(1)  # (B, 1, 288)
        x_wind = self.tcn_wind(x_wind)     # (B, D, 288)
        x_wind = x_wind[:, :, -1]          

        x_weather = weather_future.transpose(1, 2)  # (B, 8, 24)
        x_weather = self.tcn_weather(x_weather)     # (B, D, 24)
        x_weather = x_weather[:, :, -1]             # -> (B, D)

        if self.fusion_mode == 'concat':
            combined = torch.cat((x_wind, x_weather), dim=1)  # (B, 2D)
        elif self.fusion_mode == 'add':
            combined = x_wind + x_weather  

        output = self.fc(combined)         # (B, 1)
        output = self.sigmoid(output)
        return output
    
tar_model_path4 = 'wind_tcn_caiso_sigmoid_v2.pth'
tar_model4 = WindPowerPredictorTCN_V2()
state_dict = torch.load(tar_model_path4, map_location=device)
tar_model4.load_state_dict(state_dict)
tar_model4.to(device)

tar_model_list = [tar_model1, tar_model2, tar_model3, tar_model4]

## 2. PALM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle
import copy

# ----------------------------
# 物理约束计算函数
# ----------------------------
import torch

def compute_constraints(weather_future, weather_scaler):
    # 获取张量的形状
    batch_size, seq_len, feature_dim = weather_future.shape

    # 反归一化数据
    weather_future_reshaped = weather_future.view(-1, feature_dim)
    weather_future_unscaled = weather_scaler.inverse_transform(weather_future_reshaped.detach().numpy())
    weather_future_unscaled = torch.tensor(weather_future_unscaled).view(batch_size, seq_len, feature_dim)

    # 提取物理参数
    DNI = weather_future_unscaled[:, :, 1]
    Solar_Zenith_Angle = weather_future_unscaled[:, :, 4]
    DHI = weather_future_unscaled[:, :, 0]
    GHI = weather_future_unscaled[:, :, 2]
    Dew_Point = weather_future_unscaled[:, :, 3]
    Temperature = weather_future_unscaled[:, :, 7]
    Relative_Humidity = weather_future_unscaled[:, :, 6]

    # 计算物理约束
    constraint_1 = torch.abs((DNI * torch.cos(Solar_Zenith_Angle * np.pi / 180) + DHI) - GHI)
    constraint_2 = torch.abs(torch.relu(Dew_Point - Temperature))
    constraint_3 = torch.abs(
        100 * torch.exp((17.625 * Dew_Point) / (243.04 + Dew_Point)) /
        torch.exp((17.625 * Temperature) / (243.04 + Temperature)) - Relative_Humidity
    )

    constraints = torch.stack([constraint_1, constraint_2, constraint_3], dim=-1)
    return constraints


class ConstraintClassifier(nn.Module):
    def __init__(self, input_dim=72): # 72 = 24 steps * 3 constraints
        super().__init__()
        # 使用较小的隐藏层，因为经过 Max Pooling 后特征已经非常鲜明
        self.net = nn.Sequential(
            nn.Linear(3, 32), 
            nn.ReLU(),
            nn.Linear(32, 1) # 输出 Logits
        )
        
    def forward(self, x):
        # x 形状: [Batch, 72]
        # 1. 重塑为 [Batch, 24, 3] -> 24个时间步，每个步长3个物理约束
        x = x.view(x.size(0), 24, 3)
        
        # 2. 时间维度最大池化：提取 24 小时中每一个约束的最极端违背值
        # 结果形状: [Batch, 3]
        x, _ = torch.max(x, dim=1)
        
        # 3. 对数缩放：增强数值稳定性
        x = torch.log1p(x)
        
        return self.net(x)


In [ ]:
def margin_loss_fn(logits, labels, margin=10.0):
    loss_norm = torch.relu(logits[labels == 0] + margin).mean()
    loss_adv = torch.relu(margin - logits[labels == 1]).mean()
    return (loss_norm if not torch.isnan(loss_norm) else 0.0) + \
           (loss_adv if not torch.isnan(loss_adv) else 0.0)

# ---------------------------------------------------------
# 3. 诊断型训练函数
# ---------------------------------------------------------
def train_classifier(train_loader, uap_loaded_list, classifier_model, weather_scaler, device='cuda', epochs=15, learning_rate=0.0005):
    optimizer = optim.Adam(classifier_model.parameters(), lr=learning_rate)
    classifier_model.to(device)

    best_gap = -float('inf')
    best_model = None

    print(f"Starting UAP-focused training on {device}...")

    for epoch in range(epochs):
        classifier_model.train()
        all_norm_logits = []
        all_adv_logits = []
        epoch_loss = 0.0

        for _, weather_future, _ in train_loader: # 不需要 history 和 power
            batch_size = weather_future.size(0)
            weather_future = weather_future.to(device)

            x_list, y_list = [], []
            
            # (1) 正常样本
            with torch.no_grad():
                n_cons = compute_constraints(weather_future.cpu(), weather_scaler).to(device).view(batch_size, -1)
                x_list.append(n_cons)
                y_list.append(torch.zeros(batch_size, 1).to(device))

            # (2) UAP 对抗样本
            with torch.no_grad():
                for uap in uap_loaded_list:
                    u_adv = torch.clamp(weather_future + uap.to(device), 0, 1)
                    u_cons = compute_constraints(u_adv.cpu(), weather_scaler).to(device).view(batch_size, -1)
                    x_list.append(u_cons)
                    y_list.append(torch.ones(batch_size, 1).to(device))

            X = torch.cat(x_list, dim=0)
            Y = torch.cat(y_list, dim=0)

            # --- 优化 ---
            optimizer.zero_grad()
            logits = classifier_model(X)
            loss = margin_loss_fn(logits, Y, margin=15.0) # 调大 Margin
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            with torch.no_grad():
                all_norm_logits.append(logits[Y == 0].cpu())
                all_adv_logits.append(logits[Y == 1].cpu())

        # --- 统计 ---
        all_norm_logits = torch.cat(all_norm_logits)
        all_adv_logits = torch.cat(all_adv_logits)
        
        n_max = all_norm_logits.max().item()
        a_min = all_adv_logits.min().item()
        current_gap = a_min - n_max
        acc = ((all_norm_logits < 0).sum() + (all_adv_logits > 0).sum()).item() / (len(all_norm_logits) + len(all_adv_logits))

        print(f"Epoch [{epoch+1:02d}] Loss: {epoch_loss/len(train_loader):.4f} | Acc: {acc:.2%} | Gap: {current_gap:.4f}")

        if current_gap > best_gap:
            best_gap = current_gap
            best_model = copy.deepcopy(classifier_model)
            print(f"   --> Success! Best Gap improved to {best_gap:.4f}")

    return best_model

In [ ]:
selected_models = ['tcn', 'lstm']
epsilons = [('003', 0.03), ('005', 0.05), ('010', 0.10)]
uap_loaded_list = []
for suffix, epsilon_val in epsilons:
    for name in selected_models:
        pkl_path = f'uap_results_{name}_epsilon_{suffix}.pkl'
        try:
            with open(pkl_path, 'rb') as f:
                data = pickle.load(f)
                uap_np = data['uap_list'][0]
                uap_tensor = torch.tensor(uap_np).float() if not isinstance(uap_np, torch.Tensor) else uap_np.float()
                uap_loaded_list.append(uap_tensor)
        except: pass

# 实例化模型
classifier_model = ConstraintClassifier(input_dim=72)

# 开始训练
PALM = train_classifier(
    train_loader=train_loader, 
    uap_loaded_list=uap_loaded_list, 
    classifier_model=classifier_model, 
    weather_scaler=weather_scaler, 
    device='cuda', 
    epochs=10, 
    learning_rate=0.0005
)


In [ ]:
torch.save(PALM.state_dict(), 'PALM_model.pth')
PALM=ConstraintClassifier(input_dim=72)
PALM.load_state_dict(torch.load('PALM_model.pth'))
PALM.eval()

In [ ]:
import torch
import torch.nn as nn

def test_constraint_classifier_uap_only(test_loader, uap_loaded_list, classifier_model, weather_scaler, device='cuda'):
    classifier_model.eval()
    classifier_model.to(device)
    
    total_samples = 0
    correct_predictions = 0
    with torch.no_grad():
        for _, weather_future, _ in test_loader:
            batch_size = weather_future.size(0)
            weather_future = weather_future.to(device)

            n_cons = compute_constraints(weather_future.cpu(), weather_scaler).to(device)
            n_cons = n_cons.view(batch_size, -1) 

            n_logits = classifier_model(n_cons)
            n_pred = (n_logits.squeeze() > 0).float() 
            
            correct_predictions += (n_pred == 0).sum().item()
            total_samples += batch_size

            for uap in uap_loaded_list:
                uap_tensor = uap.to(device)
                attacked_weather = torch.clamp(weather_future + uap_tensor, 0, 1)
 
                a_cons = compute_constraints(attacked_weather.cpu(), weather_scaler).to(device)
                a_cons = a_cons.view(batch_size, -1)

                a_logits = classifier_model(a_cons)
                a_pred = (a_logits.squeeze() > 0).float()
                
                correct_predictions += (a_pred == 1).sum().item()
                total_samples += batch_size

    overall_acc = 100 * correct_predictions / total_samples
    print(f"\n" + "="*40)
    print(f"total_samples: {total_samples}")
    print(f"Accuracy: {overall_acc:.2f}%")
    print("="*40)

    return overall_acc

palm_acc = test_constraint_classifier_uap_only(
    test_loader=test_loader, 
    uap_loaded_list=uap_loaded_list, 
    classifier_model=PALM, 
    weather_scaler=weather_scaler, 
    device='cuda'
)